In [1]:
import pandas as pd
import numpy as np 
import math
from itertools import permutations
import random

In [2]:
df  = pd.read_csv("Data set CSV.csv")
df

,City,x,y
0,1,5.500000e-08,9.860000e-09
1,2,-2.887330e+01,-7.980000e-08
2,3,-7.929160e+01,-2.140330e+01
3,4,-1.465770e+01,-4.338960e+01
4,5,-6.474730e+01,2.189820e+01
5,6,-2.905850e+01,-4.321670e+01
6,7,-7.207850e+01,1.815810e-01
7,8,-3.603660e+01,-2.161350e+01
8,9,-5.048080e+01,7.374470e+00
9,10,-5.058590e+01,-2.158820e+01


In [3]:
class City:
    def __init__(self, n ,long, lat):
        self.name = n
        self.long = long
        self.lat = lat

In [4]:
class Chromosome:
    def __init__(self, cities , cost , fitness):
        self.cities = cities
        self.cost = cost
        self.fitness = fitness        
    

In [5]:
class Genetic_Algorithm:
    all_cities = np.array([])
    
    def __init__(self, popSize , genCount , elitismPercentage ,crossProb , MutationProb , k  ):
        
        self.popSize = popSize
        self.genCount = genCount
        self.elitismPercentage = elitismPercentage
        self.crossProb  = crossProb
        self.MutationProb = MutationProb
        self.k = k
        
    def get_cities(self):
        cities = np.array([])
                
        for index, row in df.iterrows():

            cities = np.append( cities, City(row[0] , row[1] , row[2] ) )
            
            
        return cities
    

    def permutation1(self,arr):
        if len(arr) <= 1:
            return [arr]

        permutations = []
        for i in range(len(arr)):
            first_element = arr[i]
            remaining_elements = np.concatenate((arr[:i], arr[i+1:]))
            for perm in self.permutation1(remaining_elements):
                permutations.append(np.concatenate(([first_element], perm)))

                if len(permutations) == self.popSize:
                        return permutations
        

        return permutations
    
        
    def initial_population(self , cities):

        population = np.array([])

        perm = self.permutation1(cities )
        
        i = 0
        for p in perm:
            c = Chromosome(p , self.cal_cost( p , self.dm1) , self.cal_fitness(p) )
            population = np.append(population, c)


        return population
    
        
        
    def calc_eculd(self,longCurr,latCurr,longRem,latRem):
        # calculate eculidean to remaining
        distance = math.sqrt((longCurr - longRem)**2 + (latCurr - latRem)**2)
    
        return distance

    
    def distance_matrix(self ,cities ):
    
        num_cities = len(cities) # remember to check if all_cities is a global vsriable
        dist_matrix = np.zeros((num_cities, num_cities))

        for i in range(num_cities):
            for j in range(num_cities):
                lon1 , lat1 = cities[i].long, cities[i].lat
                lon2 , lat2 = cities[j].long, cities[j].lat
                dist_matrix[i, j] = self.calc_eculd(lon1,lat1,lon2,lat2)
                
        self.dm1 = dist_matrix
        
        return dist_matrix
    
    def cal_cost(self, cities , dm):

        totaldis = 0
        
        for i in range(len(dm)-1):
            totaldis += dm[int(cities[i].name)-1 , int(cities[i+1].name)-1]
            
        
        totaldis += dm[int(cities[0].name)-1 , int(cities[len(dm)-1].name)-1]
        return totaldis
         
        
    
    def cal_fitness(self,cities):
        return 1/self.cal_cost(cities ,self.dm1 )
        
    def elitism(self,population):

        elitismCount= int(self.elitismPercentage * len(population))
        
        sorted_objects = np.array(sorted(population, key=lambda x: x.fitness))
        topFittest = sorted_objects[:elitismCount]
        
        return topFittest
    
    def k_tourment(self,population):
        
        random_chrom1 = np.random.choice(population, size=self.k, replace=False)
        
        max1 = -np.inf
        maxr1 = 0
        
        for r1 in random_chrom1 :
            
            f1 = r1.fitness
            if max1 < f1  :
                max1  = f1
                maxr1 = r1
                
        return maxr1
    
    
    def partial_crossover(self,chr1 , chr2):
        
        cc1= chr1.cities
        cc2= chr2.cities
        
        cc1_temp = chr1.cities
        cc2_temp = chr2.cities
        
        thresh1 = random.randint(0, len(cc1) - 2)
        thresh2 = random.randint(thresh1 + 1, len(cc1) - 1)
        
        for i in range(thresh1, thresh2):
            
            index1 = np.where(cc2 == cc1_temp[i])[0]
            
            if index1 != i:
                cc2[index1] = cc2[i]
                
            cc2[i] =  cc1_temp[i]
            
            
            index2 = np.where(cc1 == cc2_temp[i])[0]
            
            if index2 != i:
                cc1[index2] = cc1[i]
                
            cc1[i] =  cc2_temp[i]
            
        return cc1,cc2
    
    def mutation_swap(self, chr1):

        idx1, idx2 = np.random.choice(len(chr1), size=2, replace=False)
        chr1[idx1], chr1[idx2] = chr1[idx2], chr1[idx1]
    
        return chr1

    def cross_over_generation(self, population):
        elitismCount= int(self.elitismPercentage * len(population))
        crossover_count = int((len(population) - elitismCount)/2 )
        new_gen = np.array([])
        
        for i in range(crossover_count):
            p1 = self.k_tourment(population)
            p2 = self.k_tourment(population)
            
            random_number = random.random()
            if (random_number < self.crossProb ):
    
                c1 , c2 = self.partial_crossover(p1 , p2 )
                
                ch1 = Chromosome(c1 , self.cal_cost( c1 , self.dm1) , self.cal_fitness(c1) )
                ch2 = Chromosome(c2 , self.cal_cost( c2 , self.dm1) , self.cal_fitness(c2) )
                new_gen = np.append(new_gen, ch1 )
                new_gen = np.append(new_gen, ch2)
                
            else:
                new_gen = np.append(new_gen, p1 )
                new_gen = np.append(new_gen, p2 )
        
        return new_gen
    
                
    def mutation_generation(self,new_gen ):
                
        for i in range(len(new_gen)):
                        
            random_number = random.random()
            if (random_number < self.MutationProb ):
                
                c= self.mutation_swap(new_gen[i].cities)                
                ch = Chromosome(c , self.cal_cost( c , self.dm1) , self.cal_fitness(c) )
                new_gen[i] = ch
        
        return new_gen
    
    
    
    def termination(self , i):
        # if all visited terminate
        if i > self.genCount:
            return 1
        else:
            return 0

    def genetic_steps(self):
        
        cities = self.get_cities()
        dm = self.distance_matrix(cities)
        old_gen = self.initial_population(cities)
        m = 0
        while(True) :
            
            topFittest=self.elitism(old_gen)
            new_gen = self.cross_over_generation(old_gen)
            new_gen = self.mutation_generation(new_gen )
            old_gen = np.hstack((topFittest, new_gen))
            
            
            m+=1
            if self.termination(m):
                
                min_val = old_gen[0].cost
                min_chr = old_gen[0]
                # Iterate through the array
                for chr1 in old_gen:
                    if chr1.cost < min_val:
                        min_val = chr1.cost
                        min_chr = chr1 

                print("Maximum fitness:", 1/min_val)
                print("Minimum cost:", min_val)
                
                break
        return min_chr
        

In [34]:
genetic  = Genetic_Algorithm(50 , 100 , .02 ,0.6 , 0.1 , 5 )

In [35]:
best_route = genetic.genetic_steps()

Maximum fitness: 0.0018896722371368982
Minimum cost: 529.1923013671045


In [11]:
for b in best_route.cities: 
    print(b.name)

10.0
6.0
2.0
3.0
15.0
12.0
5.0
14.0
9.0
1.0
8.0
11.0
13.0
4.0
7.0
